In [2]:
import pandas as pd
import os
from datetime import datetime
import re
import matplotlib.pyplot as plt

In [8]:

#[INFO] [1715773048.812556027] [workspace_USB]: START	PID 3429	UID 1e790398-6269-4160-ae2e-cf1f0d9c667d	TIMESTAMP 1715773048506497093
#[INFO] [1715773049.277655391] [workspace_USB]: HAND	PID 3429	UID 1e790398-6269-4160-ae2e-cf1f0d9c667d	TIMESTAMP 1715773049274735592
#[INFO] [1715773049.303038248] [workspace_USB]: FINISH	PID 3429	UID 1e790398-6269-4160-ae2e-cf1f0d9c667d	TIMESTAMP 1715773049300404166

def process_camera_log_mc(log_file_path):
    # Read the log file and split it into lines
    with open(log_file_path, "r") as file:
        lines = file.readlines()

    # Initialize lists to store parsed data
    pid_list = []
    uid_list = []
    start_timestamp_list = []
    hand_timestamp_list = []
    finish_timestamp_list = []

    if lines and "Camera started transmitting" in lines[0]:
        lines.pop(0)

    # Iterate through lines, parsing the relevant data
    for i in range(0, len(lines), 3):
        # Check if all four lines for an entity are present
        if i + 3 < len(lines):
            start_line = lines[i].split()
            hand_line = lines[i + 1].split()
            finish_line = lines[i + 2].split()
            
            # Check if all lines correspond to the same PID
            if (start_line[7] == hand_line[7] == finish_line[7]):
                pid = start_line[5]
                uid = start_line[7]
                start_timestamp = int(start_line[9])
                hand_timestamp = int(hand_line[9])
                finish_timestamp = int(finish_line[9])

                pid_list.append(pid)
                uid_list.append(uid)
                start_timestamp_list.append(start_timestamp)
                hand_timestamp_list.append(hand_timestamp)
                finish_timestamp_list.append(finish_timestamp)

    # Create a pandas DataFrame
    data = {
        "pid": pid_list,
        "uid": uid_list,
        "start_timestamp": start_timestamp_list,
        "hand_timestamp": hand_timestamp_list,
        "finish_timestamp": finish_timestamp_list
    }

    df = pd.DataFrame(data)

    # Convert timestamps to datetime objects
    df["start_timestamp"] = pd.to_datetime(df["start_timestamp"])
    df["hand_timestamp"] = pd.to_datetime(df["hand_timestamp"])
    df["finish_timestamp"] = pd.to_datetime(df["finish_timestamp"])

    return df


def process_controller_log_mc(log_file_path):
    # Read the log file and split it into lines
    with open(log_file_path, "r") as file:
        lines = file.readlines()

    # Initialize lists to store parsed data
    space_pid_list = []
    hand_pid_list = []
    uid_list = []
    space_timestamp_list = []
    hand_timestamp_list = []
    
    # Iterate through lines, parsing the relevant data
    for i in range(0, len(lines)):
        if i + 1 < len(lines):
            line = lines[i].split()
            
            pid = line[5]
            uid = line[7]
            space_timestamp = int(line[9])
            
            space_pid_list.append(pid)
            uid_list.append(uid)
            space_timestamp_list.append(space_timestamp)
                
    # Create a pandas DataFrame
    data = {
        "space_pid": space_pid_list,
        "uid": uid_list,
        "spacestate_receive": space_timestamp_list,
    }

    df = pd.DataFrame(data)
    df.drop(df.index[0], inplace=True)
    # Convert timestamps to datetime objects
    df["spacestate_receive"] = pd.to_datetime(df["spacestate_receive"])
    return df


In [14]:
pinned = process_camera_log_mc("/home/szogabaha/src/uu/thesis/software/hilcps_edge_computing/experiments/oslevel/pinned_threads/camera.log")
unpinned = process_camera_log_mc("/home/szogabaha/src/uu/thesis/software/hilcps_edge_computing/experiments/oslevel/unpinned_threads/camera.log")

pinned["exec_time"] = pinned["finish_timestamp"] - pinned["start_timestamp"]
unpinned["exec_time"] = unpinned["finish_timestamp"] - unpinned["start_timestamp"]
pinned["exec_time"].mean()


Timedelta('0 days 00:00:00.211217717')

In [15]:
unpinned["exec_time"].mean()

Timedelta('0 days 00:00:00.222544935')